In [1]:
%load_ext autoreload
%autoreload 2
%pip install colour ruptures
import pandas as pd
import numpy as np
import bz2
import csv
import io
import json
import re
import time
import random
import requests
import datetime
from tqdm import tqdm
import json
from matplotlib import ticker
from colour import Color
import ruptures as rpt
from pathlib import Path
from pprint import pprint
from typing import List, Dict
from dateutil.relativedelta import relativedelta
import lsde2021.csv as csvutils
import lsde2021.utils as utils
import lsde2021.download as dl
import lsde2021.changepoints as cp
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,3)

     |████████████████████████████████| 971 kB 4.9 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
MAX_MEMORY = "30G"

spark = SparkSession \
    .builder \
    .appName("parse-wikipedia-sql-dumps") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .config('spark.ui.showConsoleProgress', 'false') \
    .getOrCreate()
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/03 01:23:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/03 01:23:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
languages = pd.read_csv("./data/languages.csv", index_col="code")
print(languages["group"].unique().tolist())
languages.head()

['ar', 'az', 'bg', 'bs', 'ca', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'ga', 'hi', 'he', 'hu', 'hy', 'id', 'is', 'it', 'ja', 'ko', 'ku', 'lb', 'lt', 'nl', 'no', 'pl', 'pt', 'ro', 'ru', 'sl', 'sq', 'sr', 'sv', 'tr', 'uk', 'vi', 'zh']


,name,dbname,group
code,,,
ar,Arabic,arwiki,ar
ary,Moroccan Arabic,arywiki,ar
arz,Egyptian Arabic,arzwiki,ar
az,Azerbaijani,azwiki,az
azb,South Azerbaijani,azbwiki,az


In [7]:
stringency = spark.read.format("parquet").load(f"../nvme/oxcgrt-covid-policy-tracker/OxCGRT_withnotes.parquet")
all_days = list(dl.date_range( datetime.date(2019, 1, 1), datetime.date(2019, 12, 31) ))
all_days += list(dl.date_range( datetime.date(2020, 1, 1), datetime.date(2020, 12, 31) ))
all_days += list(dl.date_range( datetime.date(2021, 1, 1), datetime.date(2021, 9, 30) ))
# print(stringency.select("CountryName").distinct().rdd.flatMap(lambda x: x).collect())

In [13]:
for country_group in languages["group"].unique().tolist():
    print(country_group)
    output_file = Path(f"../nvme/country_pageviews/pageviews_{country_group}.parquet")
    if output_file.exists():
        print(f"using existing {output_file} ...")
        continue
    
    year_df = None
    for day in tqdm(all_days):
        try:
            data = Path("../nvme/pageview_complete_per_topic") / Path("/".join(dl.wikimedia_pageview_complete_local_file(day, monthly=False))).with_suffix(".parquet")
            df = spark.read.format("parquet").load(str(data / f"group={country_group}"))
            # df = df.filter(F.col(f"level{level}_daily_total").isNotNull())
            df = df.withColumn("date", F.lit(day))
        except Exception:
            continue
            
        if year_df is None:
            year_df = df
        else:
            year_df = year_df.union(df)
    year_df.write.format("parquet").mode("overwrite").save(str(output_file))

ar
using existing ../nvme/country_pageviews/pageviews_ar.parquet ...
az
using existing ../nvme/country_pageviews/pageviews_az.parquet ...
bg
using existing ../nvme/country_pageviews/pageviews_bg.parquet ...
bs
using existing ../nvme/country_pageviews/pageviews_bs.parquet ...
ca
using existing ../nvme/country_pageviews/pageviews_ca.parquet ...
cs
using existing ../nvme/country_pageviews/pageviews_cs.parquet ...
da
using existing ../nvme/country_pageviews/pageviews_da.parquet ...
de
using existing ../nvme/country_pageviews/pageviews_de.parquet ...
el
using existing ../nvme/country_pageviews/pageviews_el.parquet ...
en
using existing ../nvme/country_pageviews/pageviews_en.parquet ...
es
using existing ../nvme/country_pageviews/pageviews_es.parquet ...
et
using existing ../nvme/country_pageviews/pageviews_et.parquet ...
fi
using existing ../nvme/country_pageviews/pageviews_fi.parquet ...
fr


100%|██████████| 1004/1004 [03:22<00:00,  4.95it/s]
21/11/03 13:29:16 WARN DAGScheduler: Broadcasting large task binary with size 11.6 MiB


ga


100%|██████████| 1004/1004 [03:17<00:00,  5.08it/s]
21/11/03 13:36:45 WARN DAGScheduler: Broadcasting large task binary with size 11.6 MiB


hi


...

100%|██████████| 1004/1004 [03:12<00:00,  5.20it/s]
21/11/03 16:08:39 WARN DAGScheduler: Broadcasting large task binary with size 11.6 MiB


vi


100%|██████████| 1004/1004 [03:12<00:00,  5.21it/s]
21/11/03 16:15:00 WARN DAGScheduler: Broadcasting large task binary with size 11.6 MiB


zh


100%|██████████| 1004/1004 [03:12<00:00,  5.21it/s]
21/11/03 16:21:02 WARN DAGScheduler: Broadcasting large task binary with size 11.6 MiB


In [ ]:
test = "Germany"
# test = "China"
test = "Italy"
country_stringency, changepoints = cp.get_change_points(stringency, test)
cp.plot_changepoints(
    country_stringency.toPandas().set_index("Date"),
    changepoints,
    title=f"Changepoints based on stringency index for {test}"
)

In [ ]:
china = stringency.filter(
    (F.lower(F.col("CountryName")) == country.lower())
)
# & (F.col("Date") == F.lit(datetime.date(2020,1,30)))
china = china.groupBy("Date", "CountryName").agg(F.mean("StringencyIndex").alias("StringencyIndex"))
china = china.select("Date", "StringencyIndex")
china = china.sort(F.col("Date").asc())
# , "CountryCode", "RegionName", "RegionCode", "Jurisdiction")
china.show()
china = china.toPandas().set_index("Date")
print(china.head())
plt.plot(china, color="blue", linestyle="-")
plt.show()